**[MDT-01]** 必要なモジュールをインポートして、乱数のシードを設定します。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160703)
tf.set_random_seed(20160703)

**[MDT-02]** MNISTのデータセットを用意します。

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
### TensorFlowのバージョンによっては下記のwarningが表示されるが、無視するしかない。
# WARNING:tensorflow:From <ipython-input-2-******>:1: ...
# Instructions for updating:
# Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
#...

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[MDT-03]** 畳込みフィルターが1層のCNNを表現するクラスを定義します。

In [3]:
class SingleCNN:
    def __init__(self, num_filters, num_units):
        with tf.Graph().as_default():
            self.prepare_model(num_filters, num_units)
            self.prepare_session()

    def prepare_model(self, num_filters, num_units):
        num_units1 = 14*14*num_filters
        num_units2 = num_units
        
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name='input')
            x_image = tf.reshape(x, [-1,28,28,1])

        with tf.name_scope('convolution'):
            W_conv = tf.Variable(
                tf.truncated_normal([5,5,1,num_filters], stddev=0.1),
                name='conv-filter')
            h_conv = tf.nn.conv2d(
                x_image, W_conv, strides=[1,1,1,1], padding='SAME',
                name='filter-output')

        with tf.name_scope('pooling'):            
            h_pool =tf.nn.max_pool(h_conv, ksize=[1,2,2,1],
                                   strides=[1,2,2,1], padding='SAME',
                                   name='max-pool')
            h_pool_flat = tf.reshape(h_pool, [-1, 14*14*num_filters],
                                     name='pool-output')

        with tf.name_scope('fully-connected'):
            w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
            b2 = tf.Variable(tf.zeros([num_units2]))
            hidden2 = tf.nn.relu(tf.matmul(h_pool_flat, w2) + b2,
                                 name='fc-output')

        with tf.name_scope('softmax'):
            w0 = tf.Variable(tf.zeros([num_units2, 10]))
            b0 = tf.Variable(tf.zeros([10]))
            p = tf.nn.softmax(tf.matmul(hidden2, w0) + b0,
                              name='softmax-output')
            
        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10], name='labels')
            loss = -tf.reduce_sum(t * tf.log(p), name='loss')
            train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
            
        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction,
                                              tf.float32), name='accuracy')
            
        tf.summary.scalar("loss", loss)
        tf.summary.scalar("accuracy", accuracy)
        tf.summary.histogram("convolution_filters", W_conv)
        
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss = loss
        self.accuracy = accuracy
        
    def prepare_session(self):
        ### >>> with GPU 80%
        #gpuConfig = tf.ConfigProto(
        #    gpu_options = tf.GPUOptions(
        #        per_process_gpu_memory_fraction = 0.8),
        #    device_count = {'GPU':0})
        #sess = tf.Session(config = gpuConfig)
        ### <<<
        #
        ### >>> with CPU only
        sess = tf.Session()
        ### <<<
        sess.run(tf.global_variables_initializer())
        ### 下記はPython3.6ではエラーとなるので、上記に変更した。
        #sess.run(tf.initialize_all_variables())
        summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter("/tmp/mnist_df_logs", sess.graph)
        
        self.sess = sess
        self.summary = summary
        self.writer = writer

**[MDT-04]** TensorBoard用のデータ出力ディレクトリーを削除して初期化しておきます。

In [4]:
### Windowsの場合、下記の行頭の「#」を削除して使用すること
!del /q \tmp\mnist_df_logs
### Linux等、Windows以外の場合、下記の行頭の「#」を削除し、上記の行頭に「#」を挿入して使用すること
#!rm -rf /tmp/mnist_df_logs

**[MDT-05]** パラメーターの最適化を4000回繰り返します。テストセットに対して約98%の正解率が得られます。

In [5]:
cnn = SingleCNN(16, 1024)

i = 0
for _ in range(4000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    cnn.sess.run(cnn.train_step, feed_dict={cnn.x:batch_xs, cnn.t:batch_ts})
    if i % 50 == 0:
        summary, loss_val, acc_val = cnn.sess.run(
            [cnn.summary, cnn.loss, cnn.accuracy],
            feed_dict={cnn.x:mnist.test.images, cnn.t:mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        cnn.writer.add_summary(summary, i)

Step: 50, Loss: 2835.226562, Accuracy: 0.917300
Step: 100, Loss: 2152.826172, Accuracy: 0.936300
Step: 150, Loss: 1854.476318, Accuracy: 0.948300
Step: 200, Loss: 1919.439087, Accuracy: 0.940800
Step: 250, Loss: 1423.470337, Accuracy: 0.958200
Step: 300, Loss: 1334.110718, Accuracy: 0.959900
Step: 350, Loss: 1268.031982, Accuracy: 0.961700
Step: 400, Loss: 1182.166382, Accuracy: 0.963700
Step: 450, Loss: 1173.881836, Accuracy: 0.963800
Step: 500, Loss: 1054.014282, Accuracy: 0.968500
Step: 550, Loss: 1014.436768, Accuracy: 0.970900
Step: 600, Loss: 967.272583, Accuracy: 0.970300
Step: 650, Loss: 909.070435, Accuracy: 0.972400
Step: 700, Loss: 911.728271, Accuracy: 0.972300
Step: 750, Loss: 870.296082, Accuracy: 0.973200
Step: 800, Loss: 909.182373, Accuracy: 0.972000
Step: 850, Loss: 803.462280, Accuracy: 0.975900
Step: 900, Loss: 829.588074, Accuracy: 0.975200
Step: 950, Loss: 930.559814, Accuracy: 0.972200
Step: 1000, Loss: 883.069458, Accuracy: 0.972000
Step: 1050, Loss: 881.012756,